# Digital Demodulation of RAW signal

All the notation & argument naming follows Blais review paper

$$
\begin{align}
\hat{V}_{I}(t) & = V_{\text{IF}}\Big[ \hat{X}_{f}(t)\cos(\omega_{\text{IF}}t)-\hat{P_{f}}(t)\sin(\omega_{\text{IF}}t)\Big] + \hat{V}_{\text{noise},t}(t) \\ \\
\hat{V}_{Q}(t) & = -V_{\text{IF}}\Big[ \hat{P}_{f}(t)\cos(\omega_{\text{IF}}t)+\hat{X_{f}}(t)\sin(\omega_{\text{IF}}t)\Big] + \hat{V}_{\text{noise},t}(t)

\end{align}
$$

$$
\begin{equation}
\hat{X}_f = \frac{\hat{a}^\dagger_f + \hat{a}_f}{2}, 
\quad 
\hat{P}_f = \frac{i(\hat{a}^\dagger_f - \hat{a}_f)}{2},
\end{equation}
$$

$$
\begin{bmatrix}
X_{f}(t) \\
P_{f}(t)
\end{bmatrix}
= R^{-1}(t)
\begin{bmatrix}
V_{I}(t) \\
V_{Q}(t)
\end{bmatrix} 

$$

In [1]:
import numpy as np 
from pprint import pprint
from copy import deepcopy


from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
#from laboneq.contrib.example_helpers.generate_device_setup import generate_device_setup
from laboneq.dsl.device import DeviceSetup




descriptor = generate_descriptor(
    #pqsc=[""], # 장비 여러개 사용시
    shfqc_6=["DEV12256"],
    number_data_qubits=6,
    multiplex=True,
    number_multiplex=6,
    include_cr_lines=True,
    include_ef_lines=True,
    get_zsync=False,  # Only set to True when using real device
    save = True,
    filename="TG_RIP",
    ip_address="192.168.0.83"
)


#descriptor
setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="192.168.0.83")
#setup
setup.instruments[0].device_options = 'SHFQC/PLUS/QC6CH'
#setup.instruments

In [2]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=False, do_emulation=False)
#session.disconnect()

[2025.11.27 19:09:33.661] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /home/yalgaeahn/JSAHN/qubit-experiment/examples/laboneq_output/log
[2025.11.27 19:09:33.663] INFO    VERSION: laboneq 25.10.2
[2025.11.27 19:09:33.664] INFO    Connecting to data server at 192.168.0.83:8004
[2025.11.27 19:09:33.766] INFO    Connected to Zurich Instruments LabOne Data Server version 25.10.1.4 at 192.168.0.83:8004
[2025.11.27 19:09:33.809] INFO    Configuring the device setup
[2025.11.27 19:09:33.839] INFO    The device setup is configured


In [3]:
from qpu_types import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters
from operations import TransmonOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform



_loaded_qubits = load_qubit_parameters(filename="latest") 
#qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 
qubits = list(_loaded_qubits.values())





qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
#quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)

# qpu.topology.add_edge("bus", "q16", "q17")
# qpu.topology.add_edge("bus", "q17", "q18")
# qpu.topology.add_edge("bus", "q18", "q19")
# qpu.topology.plot()




from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
#workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


LOADED:./qubit_parameters/20251127-1534_TG1127.json


# Resonator Spectroscopy

In [ ]:
from experiments import resonator_spectroscopy

####################################################################################
q=qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
#temp_pars.readout_resonator_frequency = 7.6283e9 #6.04036e9 #mode1
# temp_pars.readout_range_out = - 10
# temp_pars.readout_amplitude = 0.3
# temp_pars.reset_delay_length =200e-6
#temp_pars.readout_lo_frequency = 5.6e9
temporary_parameters[q.uid] = temp_pars
print(q.readout_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#frequencies = q.parameters.readout_resonator_frequency + np.linspace(-10e6,10e6,501)
frequencies = temp_pars.readout_resonator_frequency + np.linspace(-2.0e6,2.0e6,1001)
#####################################################################################
options = resonator_spectroscopy.experiment_workflow.options()
#workflow.show_fields(options)
options.count(1024)
options.update(False)
options.use_cw(False) #qop.measure 
#options.fit_complex_resonator(False)
options.fit_lorentzian(True)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
res_spec = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    options=options,
    temporary_parameters=temporary_parameters
)
res_spec_result = res_spec.run()
res_spec_analysis_result = res_spec_result.tasks["analysis_workflow"]
pprint(res_spec_analysis_result.output)

In [ ]:
from experiments import qubit_spectroscopy

###############################################################
q = qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temp_pars.spectroscopy_amplitude = 0.006 #0.003
temporary_parameters[q.uid] = temp_pars

#q.parameters.readout_amplitude = 1.0
frequencies = q.parameters.resonance_frequency_ge+ np.linspace(-2e6, 2e6, 401)
#print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = qubit_spectroscopy.experiment_workflow.options()
options.update(False)
options.count(1024)
# workflow.show_fields(options)
###################################################################
qubit_spec = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    options=options,
    temporary_parameters=temporary_parameters
)

qubit_spec_result = qubit_spec.run()
qubit_spec_result

# Dispersive Shift

In [ ]:
qubits[0].parameters

In [ ]:
from experiments import dispersive_shift

#######################################################################
q = qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
# temp_pars.drive_lo_frequency = 4.6e9
# temp_pars.readout_length = 0.8e-6
# temp_pars.readout_integration_length = 1.0e-6
temp_pars.readout_resonator_frequency = 6.6279e9 #q e 일때 res freq으로
temporary_parameters[q.uid] = temp_pars

print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = dispersive_shift.experiment_workflow.options()
options.count(1024*4)
options.update(False)
#print(workflow.show_fields(options))

###################################################################
dispersive = dispersive_shift.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=q.parameters.readout_resonator_frequency + np.linspace(-5e6,5e6,201),
    options=options,
    states='ge',
    temporary_parameters=temporary_parameters
)

dispersive_result = dispersive.run() 
print(dispersive_result.tasks['analysis_workflow'].output)

# Params

# IQ Blobs

AcquisitionType : INTEGRATION 
AveragingMode : SINGLESHOT

In [7]:

from experiments import iq_blobs

#######################################################################
q = qubits[0]

temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temporary_parameters[q.uid] = temp_pars
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = iq_blobs.experiment_workflow.options()
options.count(2**14)
#print(workflow.show_fields(options))

###################################################################
iq = iq_blobs.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    options=options,
    states='ge',
    temporary_parameters=temporary_parameters
)
iq_blob_result = iq.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
iq_blob_result.tasks['analysis_workflow'].tasks['plot_iq_blobs'].output[q.uid]
print(iq_blob_result.tasks['analysis_workflow'].output)


DRIVE range (dBm) : 10, READOUT range (dBm) : 0
[2025.11.27 19:10:33.962] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.11.27 19:10:33.962] INFO      Workflow 'iq_blobs': execution started at 2025-11-27 10:10:33.962160Z         
[2025.11.27 19:10:33.963] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.11.27 19:10:33.971] INFO    Task 'temporary_qpu': started at 2025-11-27 10:10:33.971587Z
[2025.11.27 19:10:33.972] INFO    Task 'temporary_qpu': ended at 2025-11-27 10:10:33.972799Z
[2025.11.27 19:10:33.975] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-11-27 
[2025.11.27 19:10:33.975] INFO    10:10:33.975075Z
[2025.11.27 19:10:33.976] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-11-27 10:10:33.976080Z
[2025.11.27 19:10:33.977] INFO    Task 'create_experiment': started at 2025-11-27 10:10:33.977197Z
[2025.11.27 19:10:33.983] INFO    Task 'create_experi

In [5]:
iq_blob_result

WorkflowResult(name=iq_blobs, index=())

In [6]:
# from laboneq.simple import show_pulse_sheet
# from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

# show_pulse_sheet(compiled_experiment=iq_blob_result.tasks["compile_experiment"].output,name='test', interactive=True, max_events_to_publish=10000)


# IQ Traj



To plot trajectory 
AcquisitionType : RAW   
AveragingMode : cyclic



#### Readout Pulse Shaping

rise and fall time $\tau_r$  
- $\tau_r \gg 1/\kappa$  $\implies$ adiabatic  
- $\tau_r \le 1/\kappa$  $\implies$ non-adiabatic

일반적인 steady state discrimination 이 목표라면 $\tau_r \approx 3 \sim 5 /\kappa$  


의도적으로 non-adiabatic regime 에서 fast readout도 가능

In [ ]:
from experiments import iq_traj

####################################################################################
q=qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temporary_parameters[q.uid] = temp_pars
#######################################################################
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
pprint(q.readout_parameters())
#######################################################################
options = iq_traj.experiment_workflow.options()
options.count(2**14)
options.do_analysis(True)


#print(workflow.show_fields(options))

###################################################################
iq_t = iq_traj.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    options=options,
    states='ge',
    temporary_parameters=temporary_parameters
)

iq_traj_result = iq_t.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
iq_traj_result.tasks['analysis_workflow'].tasks['plot_iq_trajectories'].output[q.uid]

#### SAVING DATA


In [ ]:
# from laboneq.simple import save, load


# #save(obj=iq_traj_result.output, filename='traj_test')

# RDATA = load(filename='traj_test')


- For $2\mu s$ acquire we have  2000ns/0.5ns=4000 sample

# Time Trace

In [ ]:
qubits[0].readout_parameters()

In [ ]:
from laboneq_applications.experiments import time_traces

q=qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
# temp_pars.readout_amplitude = 0.4
# temp_pars.drive_lo_frequency = 4.6e9
# temp_pars.readout_length = 0.8e-6
# temp_pars.readout_integration_length=1e-6
temporary_parameters[q.uid] = temp_pars
#######################################################################
pprint(q.readout_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")

cf = 50e6
#######################################################################
options = time_traces.experiment_workflow.options()
options.update(False)
options.filter_kernels(True)
options.filter_cutoff_frequency(cf)
options.granularity(1)

#print(workflow.show_fields(options))

###################################################################
raw_trace = time_traces.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    options=options,
    states='ge',
    temporary_parameters=temporary_parameters
)

raw_trace_result = raw_trace.run()
raw_trace_analysis_result = raw_trace_result.tasks['analysis_workflow'].output
print(raw_trace_analysis_result)

In [ ]:
# from laboneq.simple import show_pulse_sheet
# from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

# show_pulse_sheet(compiled_experiment=raw_trace_result.tasks["compile_experiment"].output,name='test', interactive=True)
# plot_simulation(compiled_experiment=raw_trace_result.tasks["compile_experiment"].output)

In [ ]:
import matplotlib.pyplot as plt
tr_t=raw_trace_result.tasks['analysis_workflow'].tasks['truncate_time_traces'].output
#raw_trace_result.tasks['analysis_workflow'].tasks['extract_kernels_thresholds'].output

r=tr_t["q2"]


plt.plot(r[2])

In [ ]:
import scipy as sp
krn=kernels
sos = sp.signal.butter(N=10, Wn=90e6,btype="lowpass", fs=2e9,output="sos")
fkrn=sp.signal.sosfiltfilt(sos, krn)

plt.plot(fkrn.real)
plt.plot(fkrn.imag)
# from analysis import time_traces
# time_traces.filter_integration_kernels(qubits=)



In [ ]:
raw_trace_result.tasks['analysis_workflow'].tasks["plot_kernels_fft"].output['q2']

In [ ]:
from laboneq_applications.analysis import time_traces
e_raw = raw_trace_result.output.data['q2']['result/e'].data
g_raw = raw_trace_result.output.data['q2']['result/g'].data

# Save Qubit Parameters

In [ ]:
from helper import load_qubit_parameters, save_qubit_parameters
save_qubit_parameters(qubits={ qubit.uid : qubit for qubit in qubits}, save_folder="./qubit_parameters", filename="KRISS_last")